In [2]:
import pickle
import numpy as np
from tensorflow.python.keras.models import load_model
from tensorflow.python.client import device_lib
from keras_preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu

In [16]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8306714144837443
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3129068339
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11859713732258736786
physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [3]:
curr_folder = "D:/YandexDisk/datasets/"

path_tokenizer = curr_folder + "ru-12k-tokenizer-train.pkl"

path_train_dict = curr_folder + "captions-ru-12k-train.pkl"
path_val_dict = curr_folder + "captions-ru-12k-val.pkl"

path_features_vgg16 = curr_folder + "ru-12k-features-vgg16.pkl"
path_features_inception = "D:/features-inception"

In [5]:
def load_image_features(filename, data):
    all_features = pickle.load(open(filename, 'rb'))
    features = {k: all_features[k] for k in data}

    return features

def to_lines(data):
    all_vals = list()
    for key in data.keys():
        [all_vals.append(d) for d in data[key]]

    return all_vals

def find_max_words(data):
    lines = to_lines(data)
    return max(len(l.split()) for l in lines)

def load_train_data(train_dict_path, tokenizer_path, features_path):
    with open (train_dict_path, 'rb') as f:
        out_train_dict = pickle.load(f)
    print('кол-во подписей .............. %d' % len(out_train_dict))

    out_train_features = load_image_features(features_path, out_train_dict)

    with open (tokenizer_path, 'rb') as f:
        out_tokenizer = pickle.load(f)
    out_vocab_size = len(out_tokenizer.word_index) + 1
    print('размер словаря ............... %d' % out_vocab_size)

    out_max_words = find_max_words(out_train_dict)
    print('длина предложения в словах ... %d' % out_max_words)

    return out_train_dict, out_tokenizer, out_vocab_size, out_max_words, out_train_features

### Наборы для валидации

In [15]:
val_dict, tokenizer, vocab_size, max_words, val_features = load_train_data(path_val_dict, path_tokenizer, path_features_inception)
max_words = 22

кол-во подписей .............. 2360
размер словаря ............... 21391
длина предложения в словах ... 21


# Оценка модели

После того, как модель обучены, мы можем оценить качество её предсказаний в тестовом наборе данных.

Мы оценим модель, создав описания для всех изображений в тестовом наборе данных и оценив эти прогнозы с помощью стандартной функции затрат.

Во-первых, нам нужно иметь возможность сгенерировать описание изображения, используя обученную модель. Это включает в себя передачу маркера начального описания "startseq", генерацию одного слова, а затем рекурсивный вызов модели с сгенерированными словами в качестве входных данных до тех пор, пока не будет достигнут конец маркера последовательности "endseq" или не будет достигнута максимальная длина описания.

Приведённая ниже функция реализует это поведение и генерирует текстовое описание с учётом обученной модели и заданного подготовленного изображения в качестве входных данных. Эта функция вызывает другую функцию map_int_to_word, чтобы отобразить целочисленное предсказание обратно в слово.

In [11]:
def map_int_to_word(integer, tokenizer):
    for word, idx in tokenizer.word_index.items():
        if idx == integer:
            return word

    return None

# 0 seed the generation process
# 1 iterate over the whole length of the sequence
# 2 integer encode input sequence
# 3 pad input
# 4 predict next word
# 5 convert probability to integer
# 6 map integer to word
# 7 stop if we cannot map the word
# 8 append as input for generating the next word
# 9 stop if we predict the end of the sequence
def generate_caption(model, tokenizer, image, max_words):
    in_text = 'startseq'                                    # 0

    for i in range(max_words):                              # 1
        seq = tokenizer.texts_to_sequences([in_text])[0]    # 2
        seq = pad_sequences([seq], maxlen=max_words)        # 3

        y_hat = model.predict([image,seq], verbose=0)       # 4
        y_hat = np.argmax(y_hat)                            # 5

        word = map_int_to_word(y_hat, tokenizer)            # 6
        if word is None:                                    # 7
            break

        in_text += ' ' + word                               # 8

        if word == 'endseq':                                # 9
            break

    return in_text

### BLEU

Мы сгенерируем предсказания для всех изображений в тестовом наборе данных и в наборе данных для обучения.

Приведенная ниже функция с именем evaluate_model() будет оценивать обученную модель по заданному набору подписей изображений и признаков изображений. Фактические и прогнозируемые описания собираются и оцениваются вместе с использованием оценки алгоритма BLEU, который оценивает, насколько сгенерированный текст близок к ожидаемому тексту.

BLEU используется при переводе текста для оценки переведённого текста по одному или нескольким другим переводам.

In [12]:
def evaluate_model(model, captions, images, tokenizer, max_words):
    actual, predicted = list(), list()

    for key, captions_list in captions.items():
        references = [c.split() for c in captions_list]
        y_hat = generate_caption(model, tokenizer, images[key], max_words)

        actual.append(references)
        predicted.append(y_hat.split())

    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

### Оценка VGG16

In [14]:
path_model = 'D:/models/vgg16/model-0.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.361409
BLEU-2: 0.193161
BLEU-3: 0.137490
BLEU-4: 0.055897


In [26]:
path_model = 'D:/models/vgg16/model-1.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.406490
BLEU-2: 0.224477
BLEU-3: 0.165109
BLEU-4: 0.074745


In [25]:
path_model = 'D:/models/vgg16/model-2.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.468829
BLEU-2: 0.259854
BLEU-3: 0.191766
BLEU-4: 0.092175


In [20]:
path_model = 'D:/models/vgg16/model-3.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.512229
BLEU-2: 0.289319
BLEU-3: 0.215655
BLEU-4: 0.107287


In [19]:
path_model = 'D:/models/vgg16/model-4.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.501154
BLEU-2: 0.279348
BLEU-3: 0.207381
BLEU-4: 0.103278


In [16]:
path_model = 'D:/models/vgg16/model-5.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.501107
BLEU-2: 0.275949
BLEU-3: 0.202924
BLEU-4: 0.100838


In [18]:
path_model = 'D:/models/vgg16/model-6.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.494001
BLEU-2: 0.268168
BLEU-3: 0.196033
BLEU-4: 0.095993


In [22]:
path_model = 'D:/models/vgg16/model-7.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.489138
BLEU-2: 0.265169
BLEU-3: 0.192033
BLEU-4: 0.093191


In [23]:
path_model = 'D:/models/vgg16/model-8.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.487587
BLEU-2: 0.264134
BLEU-3: 0.191545
BLEU-4: 0.091213


In [24]:
path_model = 'D:/models/vgg16/model-9.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.482318
BLEU-2: 0.260322
BLEU-3: 0.188676
BLEU-4: 0.088879


In [15]:
path_model = 'D:/models/vgg16/model-10.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.477577
BLEU-2: 0.255085
BLEU-3: 0.185947
BLEU-4: 0.088144


In [27]:
path_model = 'D:/models/vgg16/model-11.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.476834
BLEU-2: 0.254644
BLEU-3: 0.186213
BLEU-4: 0.089660


In [17]:
%%time 
path_model = 'D:/models/vgg16/model-12.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.473901
BLEU-2: 0.252494
BLEU-3: 0.183874
BLEU-4: 0.087916
Wall time: 33min


In [18]:
%%time 
path_model = 'D:/models/vgg16/model-13.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.472866
BLEU-2: 0.252232
BLEU-3: 0.183842
BLEU-4: 0.086977
Wall time: 37min 2s


In [17]:
path_model = 'D:/models/model-14.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.464693
BLEU-2: 0.247082
BLEU-3: 0.180864
BLEU-4: 0.086513


In [19]:
%%time 
path_model = 'D:/models/vgg16/model-15.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.463840
BLEU-2: 0.247353
BLEU-3: 0.181503
BLEU-4: 0.088804
Wall time: 32min 15s


In [20]:
%%time 
path_model = 'D:/models/vgg16/model-16.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.472879
BLEU-2: 0.253866
BLEU-3: 0.186495
BLEU-4: 0.091238
Wall time: 33min 36s


In [21]:
%%time 
path_model = 'D:/models/vgg16/model-17.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.461818
BLEU-2: 0.244349
BLEU-3: 0.178495
BLEU-4: 0.085330
Wall time: 32min 53s


In [22]:
%%time 
path_model = 'D:/models/vgg16/model-18.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.459094
BLEU-2: 0.242283
BLEU-3: 0.176489
BLEU-4: 0.083362
Wall time: 31min 13s


In [13]:
path_model = 'D:/models/vgg16/model-19.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.464013
BLEU-2: 0.243522
BLEU-3: 0.175339
BLEU-4: 0.082336


In [21]:
path_model = 'D:/models/vgg16/model-39.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.446779
BLEU-2: 0.227613
BLEU-3: 0.163310
BLEU-4: 0.073715


## Оценка VGG16-2

In [7]:
path_model = 'D:/models/vgg16-2/model-0.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.330890
BLEU-2: 0.132260
BLEU-3: 0.089523
BLEU-4: 0.025266


In [9]:
%%time
path_model = 'D:/models/vgg16-2/model-1.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.251889
BLEU-2: 0.133708
BLEU-3: 0.100575
BLEU-4: 0.042080
Wall time: 1h 41min 25s


In [18]:
%%time
path_model = 'D:/models/vgg16-2/model-2.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.333354
BLEU-2: 0.182158
BLEU-3: 0.135445
BLEU-4: 0.059651
Wall time: 1h 18min 58s


In [19]:
%%time
path_model = 'D:/models/vgg16-2/model-3.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.504368
BLEU-2: 0.283720
BLEU-3: 0.207341
BLEU-4: 0.100291
Wall time: 56min 24s


In [20]:
%%time
path_model = 'D:/models/vgg16-2/model-4.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.487142
BLEU-2: 0.273837
BLEU-3: 0.201340
BLEU-4: 0.096789
Wall time: 53min 13s


In [21]:
%%time
path_model = 'D:/models/vgg16-2/model-5.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.503695
BLEU-2: 0.287683
BLEU-3: 0.213201
BLEU-4: 0.106811
Wall time: 53min 24s


In [22]:
path_model = 'D:/models/vgg16-2/model-6.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.512076
BLEU-2: 0.293938
BLEU-3: 0.219297
BLEU-4: 0.110592


In [11]:
%%time
path_model = 'D:/models/vgg16-2/model-7.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.512135
BLEU-2: 0.292193
BLEU-3: 0.217427
BLEU-4: 0.111023
Wall time: 52min 39s


In [12]:
%%time
path_model = 'D:/models/vgg16-2/model-8.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.511659
BLEU-2: 0.289843
BLEU-3: 0.216488
BLEU-4: 0.110830
Wall time: 50min 47s


In [13]:
%%time
path_model = 'D:/models/vgg16-2/model-9.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.508568
BLEU-2: 0.288118
BLEU-3: 0.213113
BLEU-4: 0.107240
Wall time: 49min 30s


In [8]:
%%time
path_model = 'D:/models/vgg16-2/model-10.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.502268
BLEU-2: 0.286105
BLEU-3: 0.213294
BLEU-4: 0.107053
Wall time: 50min 25s


In [9]:
%%time
path_model = 'D:/models/vgg16-2/model-11.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.504939
BLEU-2: 0.287782
BLEU-3: 0.215184
BLEU-4: 0.108405
Wall time: 49min 49s


In [10]:
%%time
path_model = 'D:/models/vgg16-2/model-12.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.501813
BLEU-2: 0.284998
BLEU-3: 0.212785
BLEU-4: 0.106470
Wall time: 51min 14s


In [11]:
%%time
path_model = 'D:/models/vgg16-2/model-13.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.508468
BLEU-2: 0.288323
BLEU-3: 0.215613
BLEU-4: 0.109824
Wall time: 51min 34s


In [12]:
%%time
path_model = 'D:/models/vgg16-2/model-14.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.501221
BLEU-2: 0.284398
BLEU-3: 0.211127
BLEU-4: 0.105456
Wall time: 53min 59s


In [13]:
%%time
path_model = 'D:/models/vgg16-2/model-15.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.500942
BLEU-2: 0.282896
BLEU-3: 0.211010
BLEU-4: 0.106437
Wall time: 51min 42s


In [14]:
%%time
path_model = 'D:/models/vgg16-2/model-16.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.495775
BLEU-2: 0.280361
BLEU-3: 0.210769
BLEU-4: 0.108840
Wall time: 51min 23s


In [15]:
%%time
path_model = 'D:/models/vgg16-2/model-17.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.494823
BLEU-2: 0.279827
BLEU-3: 0.209732
BLEU-4: 0.107115
Wall time: 52min 35s


In [16]:
%%time
path_model = 'D:/models/vgg16-2/model-18.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.482858
BLEU-2: 0.270046
BLEU-3: 0.200387
BLEU-4: 0.099557
Wall time: 51min 12s


In [17]:
%%time
path_model = 'D:/models/vgg16-2/model-19.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.479654
BLEU-2: 0.266615
BLEU-3: 0.198103
BLEU-4: 0.098578
Wall time: 50min 57s


### Оценка Inception

In [13]:
%%time
path_model = 'D:/models/inception/model-0.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.308462
BLEU-2: 0.147342
BLEU-3: 0.102994
BLEU-4: 0.039580
Wall time: 55min 59s


In [17]:
%%time
path_model = 'D:/models/inception/model-1.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.401386
BLEU-2: 0.206048
BLEU-3: 0.148642
BLEU-4: 0.066131
Wall time: 45min 58s


In [18]:
%%time 
path_model = 'D:/models/inception/model-2.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.465643
BLEU-2: 0.249336
BLEU-3: 0.183068
BLEU-4: 0.087200
Wall time: 38min 8s


In [19]:
%%time 
path_model = 'D:/models/inception/model-3.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.507810
BLEU-2: 0.278442
BLEU-3: 0.205553
BLEU-4: 0.101472
Wall time: 32min 43s


In [20]:
%%time 
path_model = 'D:/models/inception/model-4.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.501669
BLEU-2: 0.277188
BLEU-3: 0.206544
BLEU-4: 0.104204
Wall time: 36min 6s


In [21]:
%%time 
path_model = 'D:/models/inception/model-5.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.491208
BLEU-2: 0.269639
BLEU-3: 0.200558
BLEU-4: 0.100925
Wall time: 35min 24s


In [22]:
%%time 
path_model = 'D:/models/inception/model-6.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.475721
BLEU-2: 0.259377
BLEU-3: 0.193260
BLEU-4: 0.096344
Wall time: 37min 53s


In [23]:
%%time 
path_model = 'D:/models/inception/model-7.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.478230
BLEU-2: 0.260897
BLEU-3: 0.193663
BLEU-4: 0.096543
Wall time: 38min 13s


In [24]:
%%time 
path_model = 'D:/models/inception/model-8.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.484276
BLEU-2: 0.264768
BLEU-3: 0.195059
BLEU-4: 0.096074
Wall time: 38min 30s


In [25]:
%%time 
path_model = 'D:/models/inception/model-9.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.485589
BLEU-2: 0.270567
BLEU-3: 0.203022
BLEU-4: 0.104512
Wall time: 32min 17s


In [26]:
%%time 
path_model = 'D:/models/inception/model-10.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.486937
BLEU-2: 0.271590
BLEU-3: 0.203070
BLEU-4: 0.101883
Wall time: 38min 32s


In [27]:
%%time 
path_model = 'D:/models/inception/model-11.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.484933
BLEU-2: 0.269848
BLEU-3: 0.201381
BLEU-4: 0.100716
Wall time: 38min 23s


In [28]:
%%time 
path_model = 'D:/models/inception/model-12.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.483060
BLEU-2: 0.264644
BLEU-3: 0.195287
BLEU-4: 0.097009
Wall time: 39min 9s


In [29]:
%%time 
path_model = 'D:/models/inception/model-13.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.476378
BLEU-2: 0.256270
BLEU-3: 0.188818
BLEU-4: 0.091271
Wall time: 35min 32s


In [30]:
%%time 
path_model = 'D:/models/inception/model-14.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.462925
BLEU-2: 0.241992
BLEU-3: 0.176518
BLEU-4: 0.082743
Wall time: 37min 54s


In [31]:
%%time 
path_model = 'D:/models/inception/model-15.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.478496
BLEU-2: 0.257831
BLEU-3: 0.191438
BLEU-4: 0.095683
Wall time: 37min 3s


In [32]:
%%time 
path_model = 'D:/models/inception/model-16.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.478996
BLEU-2: 0.262167
BLEU-3: 0.195365
BLEU-4: 0.097652
Wall time: 35min 54s


In [33]:
%%time 
path_model = 'D:/models/inception/model-17.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.465538
BLEU-2: 0.250085
BLEU-3: 0.184852
BLEU-4: 0.091160
Wall time: 34min


In [34]:
%%time 
path_model = 'D:/models/inception/model-18.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.471662
BLEU-2: 0.252161
BLEU-3: 0.185183
BLEU-4: 0.089931
Wall time: 35min 4s


In [35]:
%%time 
path_model = 'D:/models/inception/model-19.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.475187
BLEU-2: 0.256462
BLEU-3: 0.189255
BLEU-4: 0.093866
Wall time: 31min 31s


## Оценка Inception-2

In [14]:
%%time
path_model = 'D:/models/inception-2/model-0.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.143182
BLEU-2: 0.059739
BLEU-3: 0.041356
BLEU-4: 0.009241
Wall time: 2h 10min 37s


In [8]:
%%time
path_model = 'D:/models/inception-2/model-1.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.404583
BLEU-2: 0.192803
BLEU-3: 0.136315
BLEU-4: 0.055770
Wall time: 1h 2min


In [9]:
%%time
path_model = 'D:/models/inception-2/model-2.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.493209
BLEU-2: 0.259008
BLEU-3: 0.187620
BLEU-4: 0.086226
Wall time: 51min 12s


In [11]:
%%time
path_model = 'D:/models/inception-2/model-3.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.510724
BLEU-2: 0.280957
BLEU-3: 0.207807
BLEU-4: 0.101033
Wall time: 53min 8s


In [8]:
%%time
path_model = 'D:/models/inception-2/model-4.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.506085
BLEU-2: 0.274217
BLEU-3: 0.202119
BLEU-4: 0.098165
Wall time: 48min 9s


In [9]:
%%time
path_model = 'D:/models/inception-2/model-5.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.513603
BLEU-2: 0.282965
BLEU-3: 0.209564
BLEU-4: 0.104223
Wall time: 51min 28s


In [10]:
%%time
path_model = 'D:/models/inception-2/model-6.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.520624
BLEU-2: 0.293764
BLEU-3: 0.219084
BLEU-4: 0.111367
Wall time: 50min 17s


In [11]:
%%time
path_model = 'D:/models/inception-2/model-7.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.515529
BLEU-2: 0.289828
BLEU-3: 0.215429
BLEU-4: 0.109241
Wall time: 49min 32s


In [12]:
%%time
path_model = 'D:/models/inception-2/model-8.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.519564
BLEU-2: 0.295451
BLEU-3: 0.220886
BLEU-4: 0.113214
Wall time: 45min 9s


In [8]:
%%time
path_model = 'D:/models/inception-2/model-9.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.521632
BLEU-2: 0.297048
BLEU-3: 0.221886
BLEU-4: 0.114537
Wall time: 48min 3s


In [11]:
%%time
path_model = 'D:/models/inception-2/model-10.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.511789
BLEU-2: 0.286977
BLEU-3: 0.212708
BLEU-4: 0.106995
Wall time: 51min 58s


In [12]:
%%time
path_model = 'D:/models/inception-2/model-11.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.503644
BLEU-2: 0.280418
BLEU-3: 0.205865
BLEU-4: 0.102691
Wall time: 56min 10s


In [8]:
%%time
path_model = 'D:/models/inception-2/model-12.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.497646
BLEU-2: 0.277767
BLEU-3: 0.205065
BLEU-4: 0.101724
Wall time: 55min 5s


In [9]:
%%time
path_model = 'D:/models/inception-2/model-13.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.491665
BLEU-2: 0.274565
BLEU-3: 0.201928
BLEU-4: 0.098994
Wall time: 54min 7s


In [10]:
%%time
path_model = 'D:/models/inception-2/model-14.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.491949
BLEU-2: 0.278453
BLEU-3: 0.208454
BLEU-4: 0.104314
Wall time: 54min 45s


In [11]:
%%time
path_model = 'D:/models/inception-2/model-15.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.497122
BLEU-2: 0.279421
BLEU-3: 0.208215
BLEU-4: 0.105135
Wall time: 54min 17s


In [12]:
%%time
path_model = 'D:/models/inception-2/model-16.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.491076
BLEU-2: 0.274560
BLEU-3: 0.204636
BLEU-4: 0.104065
Wall time: 53min 2s


In [13]:
%%time
path_model = 'D:/models/inception-2/model-17.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.486741
BLEU-2: 0.273443
BLEU-3: 0.203743
BLEU-4: 0.102320
Wall time: 53min 43s


In [14]:
%%time
path_model = 'D:/models/inception-2/model-18.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.490434
BLEU-2: 0.273504
BLEU-3: 0.202311
BLEU-4: 0.101990
Wall time: 52min 3s


In [15]:
%%time
path_model = 'D:/models/inception-2/model-19.h5'
model = load_model(path_model)
evaluate_model(model, val_dict, val_features, tokenizer, max_words)

BLEU-1: 0.490287
BLEU-2: 0.274459
BLEU-3: 0.204949
BLEU-4: 0.105633
Wall time: 54min 12s
